# <a id='toc1_'></a>[ClinVar Variant Analysis](#toc0_)

The clinvar_variation_analysis notebook contains an analysis on ClinVar variant data

**Table of contents**<a id='toc0_'></a>    
- [ClinVar Variant Analysis](#toc1_)    
  - [Initialize](#toc1_1_)    
    - [Import necessary libraries](#toc1_1_1_)    
    - [Create output directory](#toc1_1_2_)    
    - [Import variant information file](#toc1_1_3_)    
  - [Add Supported Status of Variant based on in.vrs_xform_plan.policy](#toc1_2_)    
  - [Add Normalization Status of Variant based on out.errors](#toc1_3_)    
    - [Set Normalize Status of Variant as T/F](#toc1_3_1_)    
      - [Summary Table](#toc1_3_1_1_)    
  - [Create subgroups based on Variant Status](#toc1_4_)    
    - [Supported and Normalized Variants](#toc1_4_1_)    
    - [Supported and Not Normalized Variants](#toc1_4_2_)    
    - [Not Supported Variants](#toc1_4_3_)    
  - [Counting variants from each group](#toc1_5_)    
  - [Counting variant types for each group](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Initialize](#toc0_)

### <a id='toc1_1_1_'></a>[Import necessary libraries](#toc0_)

In [1]:
import ndjson
import pandas as pd
import numpy as np
from pathlib import Path
import gzip

### <a id='toc1_1_2_'></a>[Create output directory](#toc0_)

In [2]:
path = Path("variation_analysis_output")
path.mkdir(exist_ok=True)

### <a id='toc1_1_3_'></a>[Import variant information file](#toc0_)

In [3]:
with gzip.open("vi-normalized-with-liftover.jsonl.gz", "rb") as f:
    file_content = ndjson.load(f)

In [4]:
df = pd.json_normalize(file_content)

## <a id='toc1_2_'></a>[Add Supported Status of Variant based on in.issue](#toc0_)

Checking for blanks

In [5]:
df["in.issue"].value_counts()

in.issue
No viable variation members identified.                       2095
haplotype and genotype variations are not supported.          1540
sequence for accession not supported by vrs-python release     669
repeat expressions are not supported.                          474
unsupported hgvs expression.                                   457
intronic positions are not resolvable in sequence.             271
range copies are not supported.                                 95
expression contains unbalaned paretheses.                        3
Name: count, dtype: int64

In [6]:
df["in.issue"] = df["in.issue"].fillna("None")

In [7]:
df["support_status"] = df["in.issue"].copy()

df.loc[df["support_status"] == "No viable variation members identified.", "support_status"] = False
df.loc[
    df["support_status"] == "haplotype and genotype variations are not supported.",
    "support_status",
] = False
df.loc[df["support_status"] == "sequence for accession not supported by vrs-python release", "support_status"] = False
df.loc[df["support_status"] == "repeat expressions are not supported.", "support_status"] = False
df.loc[df["support_status"] == "unsupported hgvs expression.", "support_status"] = False
df.loc[df["support_status"] == "intronic positions are not resolvable in sequence.", "support_status"] = False
df.loc[df["support_status"] == "range copies are not supported.", "support_status"] = False
df.loc[
    df["support_status"] == "expression contains unbalaned paretheses.", "support_status"
] = False
df["support_status"] = df["support_status"].fillna(False).astype(bool)

In [8]:
df["support_status"].value_counts()

support_status
True     3751863
False       5604
Name: count, dtype: int64

## <a id='toc1_3_'></a>[Add Normalization Status of Variant based on out.errors](#toc0_)

The errors are stored as a list of values, some of which are strings and other of which are dictionaries (determined by whether error was handled at the level of Variation Normalizer or after the normalizer)

The "get_errors" function extracts the text error responses for better readability and ease string processing

In [9]:
import ast

def get_errors(errors: list) -> str:
    """Takes the values for the errors and represents them as a string
    :param errors: list of errors
    :return: string representing error
    """
    if pd.isna(errors):
        return "Success"

    # Parse if it's a stringified list/dict
    if isinstance(errors, str):
        try:
            errors = ast.literal_eval(errors)
        except Exception:
            return errors  # return raw string if not a valid list or dict

    errors_out = []

    # Normalize to list
    if not isinstance(errors, list):
        errors = [errors]

    for e in errors:
        if isinstance(e, str):
            errors_out.append(e)
        elif isinstance(e, dict):
            for k, v in e.items():
                if k not in ["msg", "response-errors"]:
                    continue
                if isinstance(v, str):
                    errors_out.append(v)
                elif isinstance(v, list):
                    errors_out.extend(v)  # multiple error messages
    return "; ".join(errors_out)

In [10]:
df["error_string"] = df["out.errors"].apply(get_errors)

In [11]:
df["error_string"].value_counts()

error_string
Success                                                                            3748697
Unable to get GRCh37/GRCh38 assembly for: NC_000023.9                                  422
Unable to get GRCh37/GRCh38 assembly for: NC_000024.8                                  392
Unable to get GRCh37/GRCh38 assembly for: NC_000002.10                                 305
Unable to get GRCh37/GRCh38 assembly for: NC_000009.10                                 279
                                                                                    ...   
Unable to find classification for: NC_000010.9:g.(?_62842)_(637331_658595)del            1
Unable to find classification for: NC_000012.10:g.(?_18636)_(136086_140673)del           1
Unable to find classification for: NC_000014.8:g.(66881928_?)_(?_67493801)del            1
Unable to find classification for: NC_000016.9:g.(21596299_?)_(?_21946045)del            1
Unable to find classification for: NC_000023.10:g.(77670699_?)_(?_155233731)d

There are Not Supported variants that have no error (marked as success inaccurately) because they were labeled "Not Supported" manually.

An error ("Not Supported") is entered manually for those variants so that they are not categorized as normalized

In [12]:
df.loc[
    (~df["support_status"]) & (df["error_string"] == "Success"),
    "error_string",
] = "Not Supported"

### <a id='toc1_3_1_'></a>[Set Normalize Status of Variant as T/F](#toc0_)

If an error is present, the variant was not normalized and therefore has a False Normalize Status

In [13]:
df["normalize_status"] = df["error_string"] == "Success"
df

,out,in.variation_id,in.name,in.vrs_class,in.range_copies,in.issue,in.variation_type,in.subclass_type,in.cytogenetic,in.mappings,...,out.state.type,out.state.length,out.state.sequence,out.state.repeatSubunitLength,out.extensions,in.absolute_copies,out.copies,support_status,error_string,normalize_status
0,NaN,561260,"46,XX,der(9)(q21.2,q21.2).seq[GRCh37/hg19]der(...",Not Available,[],No viable variation members identified.,Complex,SimpleAllele,9q21.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
1,NaN,221359,chr1:17555508-24706269 complex variant,Not Available,[],No viable variation members identified.,Complex,SimpleAllele,1p36.13-36.11,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
2,NaN,3897886,NM_001003722.2:c.[1807C>T];[1705C>T],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '3897886', 'rel...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
3,NaN,982523,NM_025114.4(CEP290):c.[4655AAG[2]];[6012-12T>A],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '982523', 'rela...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
4,NaN,427839,NM_023073.3(CPLANE1):c.[2080A>G];[6968A>C],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '427839', 'rela...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757462,NaN,693214,NC_012920.1(MT-CO3):m.9739C>T,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,T,NaN,NaN,NaN,NaN,True,Success,True
3757463,NaN,693231,NC_012920.1(MT-CO3):m.9813T>C,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,C,NaN,NaN,NaN,NaN,True,Success,True
3757464,NaN,693257,NC_012920.1(MT-CO3):m.9984G>A,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,A,NaN,NaN,NaN,NaN,True,Success,True
3757465,NaN,3281075,NM_080615.1:c.1078G>A,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,A,NaN,NaN,NaN,NaN,True,Success,True


#### <a id='toc1_3_1_1_'></a>[Summary Table](#toc0_)

In the table below, the cells show the number of variants with each expected behavior and how they actually ended up performing.

If a variant was in an "expected to pass" category and ends up as text, that is an instance of a normalizer failure on a supported variant

In [14]:
summary_df = (
    df[["in.variation_id", "support_status", "in.issue", "out.type"]]
    .fillna("NONE")
    .groupby(["support_status", "in.issue", "out.type"])
    .count()
    .unstack(level=2)
    .fillna(0)
    .astype(int)["in.variation_id"]
)

In [15]:
summary_df["VariantSum"] = summary_df.sum(axis=1)

In [16]:
summary_df["NormalizedSum"] = summary_df[
    ["Allele", "CopyNumberChange", "CopyNumberCount"]
].sum(axis=1)

In [17]:
summary_df["NormalizedPercent"] = (
    summary_df["NormalizedSum"] / summary_df["VariantSum"]
).apply(lambda x: f"{round(x * 100, 2)}%")

In [18]:
summary_df = summary_df.drop(["VariantSum", "NormalizedSum"], axis=1)
summary_df

out.type                                                            Allele  \
support_status in.issue                                                      
False          No viable variation members identified.                   0   
               expression contains unbalaned paretheses.                 0   
               haplotype and genotype variations are not suppo...        0   
               intronic positions are not resolvable in sequence.        0   
               range copies are not supported.                           0   
               repeat expressions are not supported.                     0   
               sequence for accession not supported by vrs-pyt...        0   
               unsupported hgvs expression.                              0   
True           None                                                3673493   

out.type                                                           CopyNumberChange  \
support_status in.issue                                                               
False          No viable variation members identified.                            0   
               expression contains unbalaned paretheses.                          0   
               haplotype and genotype variations are not suppo...                 0   
               intronic positions are not resolvable in sequence.                 0   
               range copies are not supported.                                    0   
               repeat expressions are not supported.                              0   
               sequence for accession not supported by vrs-pyt...                 0   
               unsupported hgvs expression.                                     113   
True           None                                                           32241   

out.type                                                           CopyNumberCount  \
support_status in.issue                                                              
False          No viable variation members identified.                           0   
               expression contains unbalaned paretheses.                         0   
               haplotype and genotype variations are not suppo...                0   
               intronic positions are not resolvable in sequence.                0   
               range copies are not supported.                                   0   
               repeat expressions are not supported.                             0   
               sequence for accession not supported by vrs-pyt...                0   
               unsupported hgvs expression.                                      0   
True           None                                                          39379   

out.type                                                           NONE  \
support_status in.issue                                                   
False          No viable variation members identified.             2095   
               expression contains unbalaned paretheses.              3   
               haplotype and genotype variations are not suppo...  1540   
               intronic positions are not resolvable in sequence.   271   
               range copies are not supported.                       95   
               repeat expressions are not supported.                474   
               sequence for accession not supported by vrs-pyt...   669   
               unsupported hgvs expression.                         344   
True           None                                                6750   

out.type                                                          NormalizedPercent  
support_status in.issue                                                              
False          No viable variation members identified.                         0.0%  
               expression contains unbalaned paretheses.                       0.0%  
               haplotype and genotype variations are not suppo... 

In [19]:
summary_df.to_csv("variation_analysis_output/variant_analysis_summary_df.csv")

## <a id='toc1_4_'></a>[Create subgroups based on Variant Status](#toc0_)

### <a id='toc1_4_1_'></a>[Supported and Normalized Variants](#toc0_)

In [20]:
supported_df = df.copy()

In [21]:
supported_df = supported_df.loc[
    (supported_df["support_status"] & supported_df["normalize_status"])
]
supported_df

,out,in.variation_id,in.name,in.vrs_class,in.range_copies,in.issue,in.variation_type,in.subclass_type,in.cytogenetic,in.mappings,...,out.state.type,out.state.length,out.state.sequence,out.state.repeatSubunitLength,out.extensions,in.absolute_copies,out.copies,support_status,error_string,normalize_status
30,NaN,3247636,NC_000001.10:g.(?_100381624)_(100382277_?)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1p21.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Success,True
32,NaN,2425513,NC_000001.10:g.(?_153963273)_(154580482_?)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1q21.3,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Success,True
33,NaN,1067607,NC_000001.10:g.(?_156104586)_(156108907_?)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1q22,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Success,True
34,NaN,3247983,NC_000001.10:g.(?_169446833)_(169454047_?)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1q24.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Success,True
35,NaN,3247768,NC_000001.10:g.(?_183538257)_(183543776_?)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1q25.3,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Success,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757462,NaN,693214,NC_012920.1(MT-CO3):m.9739C>T,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,T,NaN,NaN,NaN,NaN,True,Success,True
3757463,NaN,693231,NC_012920.1(MT-CO3):m.9813T>C,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,C,NaN,NaN,NaN,NaN,True,Success,True
3757464,NaN,693257,NC_012920.1(MT-CO3):m.9984G>A,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,A,NaN,NaN,NaN,NaN,True,Success,True
3757465,NaN,3281075,NM_080615.1:c.1078G>A,Allele,[],None,single nucleotide variant,SimpleAllele,NaN,[],...,LiteralSequenceExpression,NaN,A,NaN,NaN,NaN,NaN,True,Success,True


In [22]:
variation_type_count_supported_df = supported_df.value_counts(
    ["in.variation_type", "in.issue"]
).reset_index()
variation_type_count_supported_df

,in.variation_type,in.issue,count
0,single nucleotide variant,None,3401625
1,Deletion,None,160232
2,Duplication,None,73474
3,Microsatellite,None,36095
4,copy number gain,None,21484
5,copy number loss,None,20367
6,Indel,None,17038
7,Insertion,None,13019
8,Inversion,None,1401
9,Variation,None,379


In [23]:
variation_type_count_supported_df.to_csv(
    "variation_analysis_output/variation_type_count_supported_df.csv"
)

### <a id='toc1_4_2_'></a>[Supported and Not Normalized Variants](#toc0_)

In [24]:
supported_not_normalized_df = df.copy()

In [25]:
supported_not_normalized_df = supported_not_normalized_df.loc[
    (
        supported_not_normalized_df["support_status"]
        & ~supported_not_normalized_df["normalize_status"]
    )
]
supported_not_normalized_df

,out,in.variation_id,in.name,in.vrs_class,in.range_copies,in.issue,in.variation_type,in.subclass_type,in.cytogenetic,in.mappings,...,out.state.type,out.state.length,out.state.sequence,out.state.repeatSubunitLength,out.extensions,in.absolute_copies,out.copies,support_status,error_string,normalize_status
31,NaN,3068946,NC_000001.10:g.(?_1447541)_(1454371_1455520)del,CopyNumberChange,[],None,Deletion,SimpleAllele,1p36.33,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Unable to find classification for: NC_000001.1...,False
572,NaN,3768691,NC_000003.11:g.(?_47021155)_(47021403_47030158...,CopyNumberChange,[],None,Deletion,SimpleAllele,3p21.31,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Unable to find classification for: NC_000003.1...,False
968,NaN,236305,Single allele,CopyNumberChange,[],None,Deletion,SimpleAllele,5q35.2-35.3,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False
1140,NaN,3233599,NC_000007.13:g.(?_117120078)_(117120202_117144...,CopyNumberChange,[],None,Deletion,SimpleAllele,7q31.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,Unable to find classification for: NC_000007.1...,False
1382,NaN,559453,Single allele,CopyNumberCount,[],None,Deletion,SimpleAllele,9p24.3,[],...,NaN,NaN,NaN,NaN,NaN,1,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3689480,NaN,398326,NCBI36/hg18 Xp22.12(chrX:19539849-19645781)x1,CopyNumberCount,[],None,copy number loss,SimpleAllele,Xp22.12,[],...,NaN,NaN,NaN,NaN,NaN,1,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False
3689481,NaN,400163,NCBI36/hg18 Xp11.1(chrX:58066769-58095969)x1,CopyNumberCount,[],None,copy number loss,SimpleAllele,Xp11.1,[],...,NaN,NaN,NaN,NaN,NaN,1,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False
3689483,NaN,398620,NCBI36/hg18 Yq11.23(chrY:26089399-26497939)x1,CopyNumberCount,[],None,copy number loss,SimpleAllele,Yq11.23,[],...,NaN,NaN,NaN,NaN,NaN,1,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False
3689484,NaN,399800,NCBI36/hg18 Yp11.32(chrY:439455-510957)x1,CopyNumberCount,[],None,copy number loss,SimpleAllele,Yp11.32,[],...,NaN,NaN,NaN,NaN,NaN,1,NaN,True,Unable to get GRCh37/GRCh38 assembly for: NC_0...,False


In [26]:
variation_type_count_supported_not_normalized_df = (
    supported_not_normalized_df.value_counts(
        ["in.variation_type", "in.issue"]
    ).reset_index()
)
variation_type_count_supported_not_normalized_df

,in.variation_type,in.issue,count
0,copy number gain,None,3245
1,copy number loss,None,2244
2,Deletion,None,779
3,Duplication,None,407
4,single nucleotide variant,None,71
5,Indel,None,2
6,Insertion,None,1


In [27]:
variation_type_count_supported_not_normalized_df.to_csv(
    "variation_analysis_output/variation_type_count_supported_not_normalized_df.csv"
)

### <a id='toc1_4_3_'></a>[Not Supported Variants](#toc0_)

In [28]:
not_supported_df = df.copy()

In [29]:
not_supported_df = not_supported_df.loc[
    ~not_supported_df["support_status"] & ~not_supported_df["normalize_status"]
]
not_supported_df

,out,in.variation_id,in.name,in.vrs_class,in.range_copies,in.issue,in.variation_type,in.subclass_type,in.cytogenetic,in.mappings,...,out.state.type,out.state.length,out.state.sequence,out.state.repeatSubunitLength,out.extensions,in.absolute_copies,out.copies,support_status,error_string,normalize_status
0,NaN,561260,"46,XX,der(9)(q21.2,q21.2).seq[GRCh37/hg19]der(...",Not Available,[],No viable variation members identified.,Complex,SimpleAllele,9q21.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
1,NaN,221359,chr1:17555508-24706269 complex variant,Not Available,[],No viable variation members identified.,Complex,SimpleAllele,1p36.13-36.11,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
2,NaN,3897886,NM_001003722.2:c.[1807C>T];[1705C>T],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '3897886', 'rel...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
3,NaN,982523,NM_025114.4(CEP290):c.[4655AAG[2]];[6012-12T>A],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '982523', 'rela...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
4,NaN,427839,NM_023073.3(CPLANE1):c.[2080A>G];[6968A>C],Not Available,[],haplotype and genotype variations are not supp...,CompoundHeterozygote,Genotype,NaN,"[{'system': 'ClinVar', 'code': '427839', 'rela...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3689488,NaN,41607,NM_000245.2:c.-14-3312A>T,Unknown,[],No viable variation members identified.,single nucleotide variant,SimpleAllele,7q31.2,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
3689489,NaN,3764520,G317D,Unknown,[],No viable variation members identified.,single nucleotide variant,SimpleAllele,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
3689490,NaN,16606,"EPHX1, -4238T-A",Unknown,[],No viable variation members identified.,single nucleotide variant,SimpleAllele,1q42.1,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False
3689491,NaN,3901242,"MTTH, 12148T-C",Unknown,[],No viable variation members identified.,single nucleotide variant,SimpleAllele,NaN,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Supported,False


In [30]:
variation_type_count_not_supported_df = not_supported_df.value_counts(
    ["in.variation_type", "in.issue"]
).reset_index()
variation_type_count_not_supported_df

,in.variation_type,in.issue,count
0,Haplotype,haplotype and genotype variations are not supp...,617
1,Diplotype,haplotype and genotype variations are not supp...,596
2,Deletion,No viable variation members identified.,593
3,Microsatellite,repeat expressions are not supported.,456
4,Deletion,sequence for accession not supported by vrs-py...,336
5,CompoundHeterozygote,haplotype and genotype variations are not supp...,297
6,single nucleotide variant,No viable variation members identified.,290
7,Translocation,No viable variation members identified.,282
8,Insertion,No viable variation members identified.,264
9,Deletion,intronic positions are not resolvable in seque...,220


In [31]:
variation_type_count_not_supported_df.to_csv(
    "variation_analysis_output/variation_type_count_not_supported_df.csv"
)

Sanity check: making sure there are no supported variants that have been marked as normalized

In [32]:
not_supported_but_normalized_df = df.copy()

In [33]:
not_supported_but_normalized_df = not_supported_but_normalized_df.loc[
    (
        ~not_supported_but_normalized_df["support_status"]
        & not_supported_but_normalized_df["normalize_status"]
    )
]
not_supported_but_normalized_df

,out,in.variation_id,in.name,in.vrs_class,in.range_copies,in.issue,in.variation_type,in.subclass_type,in.cytogenetic,in.mappings,...,out.state.type,out.state.length,out.state.sequence,out.state.repeatSubunitLength,out.extensions,in.absolute_copies,out.copies,support_status,error_string,normalize_status


## <a id='toc1_5_'></a>[Counting variants from each group](#toc0_)

In [34]:
num_supported = len(supported_df)
num_supported_not_normalized = len(supported_not_normalized_df)
num_not_supported_but_normalized = len(not_supported_but_normalized_df)
num_not_supported = len(not_supported_df)

In [35]:
summary_df2 = pd.DataFrame(
    {
        "Supported": [num_supported, num_supported_not_normalized],
        "Not Supported": [num_not_supported_but_normalized, num_not_supported],
    }
)

In [36]:
summary_df2.index = ["Normalized", "Not Normalized"]
summary_df2

,Supported,Not Supported
Normalized,3745114,0
Not Normalized,6749,5604


## <a id='toc1_6_'></a>[Counting variant types for each group](#toc0_)

In [37]:
variation_type_count_summary_df = pd.merge(
    pd.merge(
        variation_type_count_supported_df,
        variation_type_count_supported_not_normalized_df,
        on="in.variation_type",
        how="left",
    ),
    variation_type_count_not_supported_df,
    on="in.variation_type",
    how="right",
)
variation_type_count_summary_df = variation_type_count_summary_df.replace(
    np.nan, 0, regex=True
)

In [38]:
variation_type_count_summary_df = variation_type_count_summary_df.rename(
    columns={
        "in.id_x": "supported",
        "in.id_y": "supported_not_normalized",
        "in.id": "not_supported",
    }
)

In [39]:
variation_type_count_summary_df.to_csv(
    "variation_analysis_output/variation_type_count_summary_df.csv"
)
variation_type_count_summary_df

,in.variation_type,in.issue_x,count_x,in.issue_y,count_y,in.issue,count
0,Haplotype,0,0.0,0,0.0,haplotype and genotype variations are not supp...,617
1,Diplotype,0,0.0,0,0.0,haplotype and genotype variations are not supp...,596
2,Deletion,None,160232.0,None,779.0,No viable variation members identified.,593
3,Microsatellite,None,36095.0,0,0.0,repeat expressions are not supported.,456
4,Deletion,None,160232.0,None,779.0,sequence for accession not supported by vrs-py...,336
5,CompoundHeterozygote,0,0.0,0,0.0,haplotype and genotype variations are not supp...,297
6,single nucleotide variant,None,3401625.0,None,71.0,No viable variation members identified.,290
7,Translocation,0,0.0,0,0.0,No viable variation members identified.,282
8,Insertion,None,13019.0,None,1.0,No viable variation members identified.,264
9,Deletion,None,160232.0,None,779.0,intronic positions are not resolvable in seque...,220
